# Bedrock Function Calling with Mistral models

In this Jupyter Notebook, we walkthrough an implementation of function calling and agentic workflows using Mistral models on Amazon Bedrock. Function Calling is a powerful technique that allows large language models to be decomposed into smaller, more interpretable functions, which can be combined and executed in various ways to perform complex tasks.

---
## Mistral Model Selection

Today, there are two Mistral models available on Amazon Bedrock:

### 1. Mistral 7B Instruct

- **Description:** A 7B dense Transformer model, fast-deployed and easily customizable. Small yet powerful for a variety of use cases.
- **Max Tokens:** 32K
- **Languages:** English
- **Supported Use Cases:** Text summarization, structuration, question answering, and code completion

### 2. Mixtral 8X7B Instruct

- **Description:** A 7B sparse Mixture-of-Experts model with stronger capabilities than Mistral 7B. Utilizes 12B active parameters out of 45B total.
- **Max Tokens:** 32K
- **Languages:** English, French, German, Spanish, Italian
- **Supported Use Cases:** Text summarization, structuration, question answering, and code completion

### 3. Mistral Large

- **Description:** A cutting-edge text generation model with top-tier reasoning capabilities. It can be used for complex multilingual reasoning tasks, including text understanding, transformation, and code generation.
- **Max Tokens:** 32K
- **Languages:** English, French, German, Spanish, Italian
- **Supported Use Cases:** Synthetic Text Generation, Code Generation, RAG, or Agents

### Performance and Cost Trade-offs

The table below compares the model performance on the Massive Multitask Language Understanding (MMLU) benchmark and their on-demand pricing on Amazon Bedrock.

| Model           | MMLU Score | Price per 1,000 Input Tokens | Price per 1,000 Output Tokens |
|-----------------|------------|------------------------------|-------------------------------|
| Mistral 7B Instruct | 62.5%      | \$0.00015                    | \$0.0002                      |
| Mixtral 8x7B Instruct | 70.6%      | \$0.00045                    | \$0.0007                      |
| Mistral Large | 81.2%      | \$0.008                   | \$0.024                     |

For more information, refer to the following links:

1. [Mistral Model Selection Guide](https://docs.mistral.ai/guides/model-selection/)
2. [Amazon Bedrock Pricing Page](https://aws.amazon.com/bedrock/pricing/)


---
## Supported papameters

The Mistral AI models have the following inference parameters.


```
{
    "prompt": string,
    "max_tokens" : int,
    "stop" : [string],    
    "temperature": float,
    "top_p": float,
    "top_k": int
}
```

The Mistral AI models have the following inference parameters:

- **Temperature** - Tunes the degree of randomness in generation. Lower temperatures mean less random generations.
- **Top P** - If set to float less than 1, only the smallest set of most probable tokens with probabilities that add up to top_p or higher are kept for generation.
- **Top K** - Can be used to reduce repetitiveness of generated tokens. The higher the value, the stronger a penalty is applied to previously present tokens, proportional to how many times they have already appeared in the prompt or prior generation.
- **Maximum Length** - Maximum number of tokens to generate. Responses are not guaranteed to fill up to the maximum desired length.
- **Stop sequences** - Up to four sequences where the API will stop generating further tokens. The returned text will not contain the stop sequence.

---

## Requirements

In [ ]:
%%writefile requirements.txt
langchain==0.1.17
langchain-experimental==0.0.57
boto3==1.34.97
sqlalchemy==2.0.29
pandas==2.2.2

In [ ]:
!pip install -U -r requirements.txt --quiet

In [ ]:
pip install langchain-aws --quiet


---

In [1]:
instruct_mistral7b_id = "mistral.mistral-7b-instruct-v0:2"
instruct_mixtral8x7b_id = "mistral.mixtral-8x7b-instruct-v0:1"
mistral_large_2402_id = "mistral.mistral-large-2402-v1:0"

DEFAULT_MODEL = instruct_mistral7b_id

## Function Calling

Function calling is the ability to reliably connect a large language model (LLM) to external tools and enable effective tool usage and interaction with external APIs. Mistral models provide the ability for building LLM powered chatbots or agents that need to retrieve context for the model or interact with external tools by converting natural language into API calls to retrieve specific domain knowledge. From conversational agents and math problem solving to API integration and information extraction, multiple use cases can benefit from this capability provided by Mistral models

---

## Load data

Let's say, we have a transactional dataset tracking customers, payment amounts, payment dates, and whether payments have been fully processed for each transaction identifier

In [2]:
import pandas as pd


def load_data()-> pd.DataFrame:
    """
    Load data from a JSON file into a Pandas DataFrame.

    Returns:
        pd.DataFrame: A Pandas DataFrame containing the data loaded from the JSON file.
        If an error occurs during file loading, an error message is returned instead.

    """
    local_path = "sample_data/transactions.json"
    
    try:
        df = pd.read_json(local_path)
        return df
    except (FileNotFoundError, ValueError) as e:
        return  f"Error: {e}"

In [3]:
df = load_data()
print(df)

  transaction_id customer_id  payment_amount payment_date payment_status
0          T1001        C001          125.50   2021-10-05           Paid
1          T1002        C002           89.99   2021-10-06         Unpaid
2          T1003        C003          120.00   2021-10-07           Paid
3          T1004        C002           54.30   2021-10-05           Paid
4          T1005        C001          210.20   2021-10-08        Pending


## Initialize Bedrock with LangChain

In [4]:
from langchain_aws import ChatBedrock

In [5]:
llm = ChatBedrock(
    model_id=DEFAULT_MODEL,
    model_kwargs={"temperature": 0.1},
)

## Tools

LangChain Tools are interfaces that allow agents, chains, or language models to interact with the world. They typically include a name, description, JSON schema for input parameters, and the function to call. This information is used to prompt the language model on how to specify and take actions. We will use LangChain Tools to quickly transform our functions as a prompt that can be used by Mistral to execute function calls.

---

In [6]:
import json
from langchain.tools import tool
from pydantic.v1 import BaseModel, Field

Let’s consider we have two functions as our two tools: **retrieve_payment_status** and **retrieve_payment_date** to retrieve payment status and payment date given a transaction ID.

---

In [7]:
class Params(BaseModel):
    transaction_id: str = Field(..., description='Transaction ID')

### Function Call 1: Retrieve payment status

In [8]:
@tool
def retrieve_payment_status(params: Params) -> str:
    "Get payment status of a transaction"
    data = load_data()

    try:
        # Attempt to retrieve the payment status for the given transaction ID
        status = data[data.transaction_id == params.transaction_id].payment_status.item()
    except ValueError:
        # If the transaction ID is not found, return an error message
        return {'error': f"Transaction ID {params.transaction_id} not found."}

    # Retrieve the payment status for the corresponding index
    return json.dumps({'status': f"{status}"})

### Function Call 2: Retrieve payment date

In [9]:
@tool
def retrieve_payment_date(params: Params) -> str:
    "Get payment date of a transaction"
    data = load_data()
    
    try:
        # Attempt to retrieve the payment date for the given transaction ID
        date = data[data.transaction_id == params.transaction_id].payment_date.item()
    except ValueError:
        # If the transaction ID is not found, return an error message
        return {'error': f"Transaction ID {params.transaction_id} not found."}
    
    return json.dumps({'date': date})

### Functions

In this step, we will utilize another function from the LangChain library to transform a raw function or class into a format that can be easily understood and processed by a large language model.

In [10]:
from langchain.chains.openai_functions import convert_to_openai_function as convert_to_llm_fn
tools = [retrieve_payment_status, retrieve_payment_date]
functions = [convert_to_llm_fn(f) for f in tools]

In [11]:
functions

[{'name': 'retrieve_payment_status',
  'description': 'retrieve_payment_status(params: __main__.Params) -> str - Get payment status of a transaction',
  'parameters': {'type': 'object',
   'properties': {'params': {'type': 'object',
     'properties': {'transaction_id': {'description': 'Transaction ID',
       'type': 'string'}},
     'required': ['transaction_id']}},
   'required': ['params']}},
 {'name': 'retrieve_payment_date',
  'description': 'retrieve_payment_date(params: __main__.Params) -> str - Get payment date of a transaction',
  'parameters': {'type': 'object',
   'properties': {'params': {'type': 'object',
     'properties': {'transaction_id': {'description': 'Transaction ID',
       'type': 'string'}},
     'required': ['transaction_id']}},
   'required': ['params']}}]

## Agentic Workflow Orchestration 

In this section, we defined helper functions to automate the process of identifying the **function call** to be used by the LLM, extract the function from its response, and execute the function.

In [12]:
import re
from functools import partial

Helper Function: Extract function call representations enclosed within XML tags (`<functioncall>` and `<multiplefunctions>`)

In [13]:
def extract_function_calls(completion: str):
    if isinstance(completion, str):
        content = completion
    else:
        content = completion.content

    # Multiple functions lookup
    mfn_pattern = r"<multiplefunctions>(.*?)</multiplefunctions>"
    mfn_match = re.search(mfn_pattern, content, re.DOTALL)

    # Single function lookup
    single_pattern = r"<functioncall>(.*?)</functioncall>"
    single_match = re.search(single_pattern, content, re.DOTALL)
    
    functions = []
    
    if not mfn_match and not single_match:
         # No function calls found
        return None
    elif mfn_match:
        # Multiple function calls found
        multiplefn = mfn_match.group(1)
        for fn_match in re.finditer(r"<functioncall>(.*?)</functioncall>", multiplefn, re.DOTALL):
            fn_text = fn_match.group(1)
            try:
                functions.append(json.loads(fn_text.replace('\\', '')))
            except json.JSONDecodeError:
                pass  # Ignore invalid JSON
    else:
        # Single function call found
        fn_text = single_match.group(1)
        try:
            functions.append(json.loads(fn_text.replace('\\', '')))
        except json.JSONDecodeError:
            pass  # Ignore invalid JSON
    return functions

Helper Function: Execute function call with the arguments captured by the LLM during the AI/user interaction

In [14]:
def execute_function(function_list: list):
    for function_dict in function_list:
        function_name = function_dict['name']
        arguments = function_dict['arguments']
        
        # Check if the function exists in the current scope
        if function_name in globals():
            func = globals()[function_name]
            
            # Call the function with the provided arguments
            result = func.invoke(input=arguments)
            return result
        else:
            return {'error': f"Function '{function_name}' not found."}

Helper Function: The agentic workflow function contains the logic for orchestrating the execution of function calls

In [15]:
def run_agentic_workflow_(prompt: str, model: str, functions: list):
    # Define the function call format
    fn = """{"name": "function_name", "arguments": {"arg_1": "value_1", "arg_2": value_2, ...}}"""

    # Prepare the function string for the system prompt
    fn_str = "\n".join([str(f) for f in functions])
    
    # Define the system prompt
    system_prompt = f"""
You are a helpful assistant with access to the following functions:

{fn_str}

To use these functions respond with:

<multiplefunctions>
    <functioncall> {fn} </functioncall>
    <functioncall> {fn} </functioncall>
    ...
</multiplefunctions>

Edge cases you must handle:
- If there are no functions that match the user request, you will respond politely that you cannot help.
- If the user has not provided all information to execute the function call, ask for more details. Only, respond with the information requested and nothing else.
- If asked something that cannot be determined with the user's request details, respond that it is not possible to fullfill the request and explain why.
"""
    # Prepare the messages for the language model
    messages = [
            ("system", system_prompt),
            ("user", prompt),
    ]
    
    # Invoke the language model and get the completion
    completion = llm.invoke(messages)
    content = completion.content.strip()

    # Extract function calls from the completion
    functions = extract_function_calls(content)

    if functions:
        # If function calls are found, execute them and return the response
        fn_response = execute_function(functions)
        return fn_response
    else:
        # If no function calls are found, return the completion content
        return content

In [16]:
generation_func = partial(run_agentic_workflow_, model=llm, functions=functions)

## Q&A: Agentic Worklflow

Here, we defined a prompt to handle the conversation history and answer follow-up questions withi the agentic workflow.

In [17]:
conv_history_prompt = """
#############
Chat History:
{chat_history}
#############

You are an AI assistant designed to human-like responses based on the transaction details provided to you.

Transaction details:
{transaction_details}

Provide clear and concise responses based solely on the data, without making any assumptions or inferences beyond what is contained in the transaction details. 
If the transaction details shows an 'error' message, it means we do not have enough information. In this case, respond that it is not possible to fullfill the request and explain why"""

Next, the following function allows a user to have a conversation with Mistral models, where Mistral generates responses based on the user's input and some action or function call is executed to perform actions on your behalf. 

In [18]:
def run_qa_agent():
    # Initialize conversation history
    conversation_history = []
    
    print("Welcome to the LLM Conversation! Type 'exit' to end the conversation.")
    
    while True:
        # Get user input
        user_input = input("Ask a question: \n")
    
        # Check if the user wants to exit
        if user_input.lower() == "exit":
            print("Goodbye!")
            break
    
        # Add user input to the conversation history
        conversation_history.append(("user", user_input))
    
        # Prepare the prompt from the conversation history
        prompt = "\n".join([q[1] for q in conversation_history if 'user' in q])
    
        # Generate the action to take based on the detected function call
        action_response = generation_func(prompt)
    
        # Prepare the question-answer prompt
        qa_prompt = conv_history_prompt.format(chat_history=action_response, transaction_details=prompt)
    
        # Prepare the messages for final LLM response
        messages = [
            ("system", qa_prompt),
            ("user", str(action_response)),
        ]
    
        # Get the response from the LLM
        response = llm.invoke(messages).content.strip()

        # Display the LLM response
        print("Answer:\n", response)

In [19]:
run_qa_agent()

Welcome to the LLM Conversation! Type 'exit' to end the conversation.


Ask a question: 
 What's the status of my transaction?


Answer:
 I'm sorry, but I cannot provide the status of your transaction as the transaction ID you provided was not found in our system.


Ask a question: 
 My transaction ID is T1001


Answer:
 Based on the transaction details provided, your transaction with ID T1001 has been marked as "Paid". This means that the payment associated with this transaction has been successfully processed. If you have any further questions or need assistance with anything else, feel free to ask.


Ask a question: 
 exit


Goodbye!


In [20]:
run_qa_agent()

Welcome to the LLM Conversation! Type 'exit' to end the conversation.


Ask a question: 
 On what date was my transaction ID made?


Answer:
 I'm sorry, but we cannot fulfill your request as the transaction ID "your_transaction_ID" was not found in our system.


Ask a question: 
 My transaction ID is T1001


Answer:
 Based on the transaction details provided, the transaction ID T1001 was made on the date of October 5, 2021.


Ask a question: 
 exit


Goodbye!


## Distributors
- Amazon Web Services
- Mistral AI

---